In [1]:
import pandas as pd
import string
import nltk
import math
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex as re
import operator
import os
import sys
import time
import json
import csv
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)

In [18]:
dicoTfidf = {}
sys.path.append("C:/Users/Admin/Projet FeelCrypt/FeelCrypt/Valentin/")
import dataset_v
df = dataset_v.get_labeled_dataset(all_files = True)

Number of files loaded :  665


In [19]:
df.head()

,text,label,date,score,nb_replies,stickied
0,Recent comment by him with regards to Gavin An...,1,2015-12-15,97,3,False
1,I'm glad your trolling has come to an end,1,2015-12-15,-46,1,False
2,r/bitcoinall is great,1,2015-12-15,7,0,False
3,/r/bitcoinxt ftw!,1,2015-12-15,20,1,False
4,> their passion is financially incentivised.\r...,1,2015-12-15,9,0,False


In [22]:
start_time = time.time()
dico = { "date" : [], "words" : [] }
index = 0

# parcourir toutes les lignes du dataframe
for date in df.date.unique()[:1]:
    dicoDate = []
    scores = []
    #print(f'date = {date} | index {index}')
    # Si on est sur la même date que celle recupéré par la boucle for, on continue la tfidf actuelle
    while df["date"][index] == date and index < len(df)-1:
        # on rentre au moins une fois dedans
        if(df["score"][index] > 0):
            dicoDate.append(df["text"][index])
            scores.append(df["score"][index])
        index = index + 1
    
    if len(dicoDate) > 0:
        # On a récupéré tous les tweets de la date que l'on veut analyser avec tfidf
        # On le convertit en Series d'abord
        dicoDate = pd.Series(dicoDate)
        # len file
        len_file = len(dicoDate)
    
        # remove comments with low scores
        scoreSort = np.sort(scores)
        Q1 = scores[int(len_file/4)]
        # Keep all index of comments to remove because of their lower score
        scoreIndexToRemove = []
        cpt = 0
    
        for score in scores:
            if score < Q1:
                scoreIndexToRemove.append(cpt)
            cpt += 1
    
        
        dicoDateNew = dicoDate.drop(scoreIndexToRemove)
        # Preprocessing des données
        dicoDateNew = preprocessing(dicoDateNew, len_file)

        # maintenant on peut appliquer tfidf
        uniqueWords = []
        for sentence in dicoDateNew:
            words = [word for word in sentence]
            for word in words:
                if word not in uniqueWords:
                    uniqueWords.append(word)
        uniqueWords.sort()
    
        numOfWords = dict.fromkeys(uniqueWords, 0)
        for i in range(len_file):
            try :
                for words in dicoDateNew[i]:
                    numOfWords[words] += 1
            except KeyError:
                i += 1
    
        tf = computeTF(dicoDateNew, uniqueWords)
        idfs = dict.fromkeys(uniqueWords, 0)
        idfs = computeIDF(dicoDateNew, len_file, idfs, numOfWords)
        tfidf = computeTF_IDF(tf, idfs)
    
        # On recupere les mots les plus importants
        columns = tfidf.columns.values
        dicoTFIDFWords = {}
        indexes = 0
        for label in columns:
            dicoTFIDFWords[label] = sum(tfidf[columns[indexes]])
            indexes = indexes + 1
        dicoTFIDFWords = sorted(dicoTFIDFWords.items(), key=operator.itemgetter(1), reverse=True)
        dicoMostImportantWords = dicoTFIDFWords[0:50]
        dicoWordsToKeep = [l[0] for l in dicoMostImportantWords]
        dico["date"].append(date)
        dico["words"].append(dicoWordsToKeep)
        print(date)
print("Temps d execution tfidf : %s secondes ---" % (time.time() - start_time)) 

2015-12-15
Temps d execution tfidf : 2.7975144386291504 secondes ---


In [21]:
pd.DataFrame(dico).to_csv('TFIDF.csv')

In [13]:
dico

{'date': ['2015-12-15',
  '2015-12-16',
  '2015-12-17',
  '2015-12-30',
  '2015-12-31',
  '2016-01-01',
  '2016-01-18',
  '2016-01-19',
  '2016-01-20',
  '2016-01-21',
  '2016-01-25',
  '2016-01-26',
  '2016-01-27',
  '2016-02-01',
  '2016-02-08',
  '2016-02-11',
  '2016-02-12',
  '2016-02-14',
  '2016-02-25',
  '2016-02-26',
  '2016-02-27',
  '2016-02-28',
  '2016-02-29',
  '2016-03-01',
  '2016-03-02',
  '2016-03-03',
  '2016-03-04',
  '2016-03-05',
  '2016-03-06',
  '2016-03-12',
  '2016-03-13',
  '2016-03-14',
  '2016-03-15',
  '2016-03-18',
  '2016-05-02',
  '2016-05-03',
  '2016-05-04',
  '2016-05-05',
  '2016-05-06',
  '2016-05-07',
  '2016-05-08',
  '2016-06-30',
  '2016-07-01',
  '2016-07-02',
  '2016-07-03',
  '2016-07-13',
  '2016-07-26',
  '2016-09-22',
  '2016-11-14',
  '2016-11-15'],
 'words': [['yes',
   'mod',
   'bullshit',
   'btc',
   'post',
   'lol',
   'disinformation',
   'community',
   'question',
   'mods',
   'bitcoin',
   'btcdrak',
   'users',
   'rules',
 

## Data Cleaning

In [8]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [9]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [10]:
def remove_links(text):
    words = [w for w in text if not len(w) > 20]
    return words

In [11]:
def stem_sentence(sentence):
    porter = PorterStemmer()
    for i in range(len(sentence)):
        word = porter.stem(sentence[i])
        sentence[i] = word
    return sentence

In [12]:
# Remove all "/u/", "/r", "/r", "\n" and numbers
def preprocessing(commentArray, len_file):
    for i in range(len_file):
        try:
            commentArray[i] = re.sub(r"(\/\w\/|\w\/)", "", commentArray[i])
            commentArray[i] = re.sub(r"\\n?t?", "", commentArray[i])
            commentArray[i] = re.sub(r"[0-9]*", "", commentArray[i])
        except KeyError:
            continue
    # punctuation
    commentArray = commentArray.apply(lambda x: remove_punctuation(x))
    tokenizer = RegexpTokenizer(r'\w+')
    commentArray = commentArray.apply(lambda x : tokenizer.tokenize(x.lower()))
    
    # remove small sentence
    commentArray = commentArray[commentArray.map(len) > 2]
    
    # remove small words
    for i in range(len_file):
        try :
            for word in commentArray[i]:
                if len(word) < 3:
                    commentArray[i].remove(word)
        except KeyError :
            continue
    
    # remove stopwords
    commentArray = commentArray.apply(lambda x : remove_stopwords(x))

    # reddit link
    commentArray = commentArray.apply(lambda x : remove_links(x))
    
    return commentArray

### Term Frequency (TF)

In [13]:
def computeTF(file, uniqueWords):
    tf = []
    index = 0
    
    try :
        for sentence in file:
            tf.append(dict.fromkeys(uniqueWords, 0))
            for word in sentence:
                tf[index][word] += 1 / len(sentence)
            index += 1
    except KeyError :
        print("error")
    return tf

### Inverse Data Frequency (IDF)

In [14]:
def computeIDF(file, len_file, idfs, numOfWords):
    # Pour chaque doc, pour chaque mot on regarde sa tf
    for x, y in idfs.items():
        count = 0
        for i in range(len_file):
            try:
                for word in file[i]:
                    if x == word:
                        count += 1
                        break
            except KeyError:
                i += 1
        if count != 0:
            idfs[x] = math.log(numOfWords[x] / count)
    return idfs

### TF-IDF

In [15]:
def computeTF_IDF(tf, idfs):
    tab = []
    for i in range(len(tf)):
        dico = {}
        for mot in tf[i]:
            dico[mot] = tf[i][mot] * idfs[mot]
        tab.append(dico)
    tfidf = pd.DataFrame(tab)
    return tfidf